In [2]:
import pandas as pd
import numpy as np
import re
import glob
%matplotlib inline

In [3]:
#zuerst test mit einem der 17 files
path = "Erstsprache/Dateien/"

In [4]:
df = pd.read_excel(path + "Sprachen/Erstsprache nach Schulstufe_10_15_2019.xls", skiprows=[0,1,2,3])

In [5]:
df.head(2)

,Jahr,Schulgemeinde,Erstsprache,KG,PS,PS Sonder,Sek A,Sek B,Sek C,Sek Sonder,Total
0,2017,Adlikon (Primar),Deutsch,0.0,12.0,0.0,NaN,NaN,NaN,NaN,12
1,2017,Adlikon (Primar),Kurdisch,0.0,2.0,0.0,NaN,NaN,NaN,NaN,2


In [6]:
len(df)

4597

In [7]:
#sämtliche files mit glob reinholen
filenames = glob.glob(path + "Sprachen/*.xls")

In [8]:
#zahl der files testen
len(filenames)

18

In [9]:
filenames[0:2]

['Erstsprache/Dateien/Sprachen/Erstsprache nach Schulstufe_10_15_2019 (15).xls',
 'Erstsprache/Dateien/Sprachen/Erstsprache nach Schulstufe_10_15_2019 (1).xls']

In [10]:
dfs = []

In [11]:
#einlesen mit list comprehension, erste vier zeilen skippen
dfs = [pd.read_excel(filename, skiprows=[0,1,2,3]) for filename in filenames]

In [12]:
#test, resp. durch die einzelnen dataframes gehen
dfs[4].head(10)

,Jahr,Schulgemeinde,Erstsprache,KG,PS,PS Sonder,Sek A,Sek B,Sek C,Sek Sonder,Total
0,2004,Adlikon (Primar),Deutsch,0.0,15.0,0.0,NaN,NaN,NaN,NaN,15
1,2004,Adliswil (Einheitsgemeinde),Afrikanische Sprachen,0.0,1.0,3.0,0.0,0.0,0.0,0.0,4
2,2004,Adliswil (Einheitsgemeinde),Albanisch,24.0,58.0,13.0,7.0,15.0,7.0,4.0,128
3,2004,Adliswil (Einheitsgemeinde),Arabisch,2.0,2.0,2.0,0.0,1.0,1.0,1.0,9
4,2004,Adliswil (Einheitsgemeinde),Bosnisch,5.0,6.0,0.0,0.0,0.0,0.0,0.0,11
5,2004,Adliswil (Einheitsgemeinde),Bulgarisch,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
6,2004,Adliswil (Einheitsgemeinde),Chinesisch,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
7,2004,Adliswil (Einheitsgemeinde),Deutsch,129.0,495.0,33.0,125.0,96.0,6.0,2.0,886
8,2004,Adliswil (Einheitsgemeinde),Englisch,5.0,19.0,0.0,7.0,2.0,0.0,0.0,33
9,2004,Adliswil (Einheitsgemeinde),Finnisch,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1


In [13]:
#alle dateien zu einem datenframe zusammenhängen
df_all = pd.concat(dfs, ignore_index=True)

In [14]:
df_all.shape

(74747, 11)

In [15]:
df_all.head(10)

,Jahr,Schulgemeinde,Erstsprache,KG,PS,PS Sonder,Sek A,Sek B,Sek C,Sek Sonder,Total
0,2002,Adlikon (Primar),Deutsch,0.0,15.0,0.0,NaN,NaN,NaN,NaN,15
1,2002,Adliswil (Einheitsgemeinde),Afrikanische Sprachen,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3
2,2002,Adliswil (Einheitsgemeinde),Albanisch,12.0,63.0,10.0,4.0,8.0,2.0,4.0,103
3,2002,Adliswil (Einheitsgemeinde),Andere asiatische Sprachen,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2
4,2002,Adliswil (Einheitsgemeinde),Arabisch,1.0,3.0,2.0,1.0,0.0,0.0,1.0,8
5,2002,Adliswil (Einheitsgemeinde),Bosnisch,3.0,3.0,1.0,0.0,0.0,0.0,0.0,7
6,2002,Adliswil (Einheitsgemeinde),Chinesisch,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2
7,2002,Adliswil (Einheitsgemeinde),Deutsch,180.0,512.0,33.0,126.0,109.0,6.0,8.0,974
8,2002,Adliswil (Einheitsgemeinde),Englisch,5.0,16.0,1.0,3.0,0.0,0.0,0.0,25
9,2002,Adliswil (Einheitsgemeinde),Finnisch,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1


In [17]:
df_all.dtypes

Jahr               int64
Schulgemeinde     object
Erstsprache       object
KG               float64
PS               float64
PS Sonder        float64
Sek A            float64
Sek B            float64
Sek C            float64
Sek Sonder       float64
Total              int64
dtype: object

In [22]:
#wie heissen die schulgemeinden?
df_all["Schulgemeinde"].unique()

array(['Adlikon (Primar)', 'Adliswil (Einheitsgemeinde)',
       'Aesch (Primar)', 'Aeugst a.A. (Primar)',
       'Affoltern a.A. (Primar)', 'Affoltern a.A.-Aeugst (Sekundar)',
       'Altikon (Primar + Pol. Gem.)', 'Andelfingen (Primar)',
       'Andelfingen (Sekundar)', 'Bachenbülach (Primar)',
       'Bachs (Primar)', 'Bäretswil (Einheitsgemeinde)',
       'Bassersdorf (Einheitsgemeinde)', 'Bauma (Einheitsgemeinde)',
       'Benken (Primar)', 'Berg a.I. (Primar)',
       'Bertschikon (Primar + Pol. Gem.)', 'Birmensdorf (Primar)',
       'Birmensdorf-Aesch (Sekundar)', 'Bonstetten (Primar)',
       'Bonstetten (Sekundar)', 'Boppelsen (Primar)',
       'Brütten (Einheitsgemeinde)', 'Bubikon (Primar + Sekundar)',
       'Buch a.I. (Primar)', 'Buchs (Primar)',
       'Bülach (Primar + Pol. Gem.)', 'Bülach (Sekundar)',
       'Dachsen (Primar)', 'Dägerlen (Primar)',
       'Dällikon (Primar + Pol. Gem.)', 'Dänikon-Hüttikon (Primar)',
       'Dättlikon (Einheitsgemeinde)', 'Dielsdorf (Pri

In [40]:
#welche sprachen gibt es?
df_all["Erstsprache"].unique()

array(['Deutsch', 'Afrikanische Sprachen', 'Albanisch',
       'Andere asiatische Sprachen', 'Arabisch', 'Bosnisch', 'Chinesisch',
       'Englisch', 'Finnisch', 'Französisch', 'Griechisch',
       'Iranische Sprachen', 'Italienisch', 'Kroatisch', 'Kurdisch',
       'Mazedonisch', 'Nicht erhoben (nur tertiär)', 'Niederländisch',
       'Norwegisch', 'Polnisch', 'Portugiesisch', 'Rätoromanisch',
       'Rumänisch', 'Russisch', 'Schwedisch', 'Serbisch', 'Slowenisch',
       'Spanische Sprachen', 'Südslawisch', 'Tamil', 'Thai',
       'Tschechisch', 'Türkisch', 'Übrige europäische Sprachen',
       'Übrige indoari. und drawid.Spr', 'Übrige ostasiatische Sprachen',
       'Übrige slawische Sprachen', 'Übrige Sprachen', 'Dänisch',
       'Slowakisch', 'Übrige afrikanische Sprachen',
       'Übrige osteuropäische Sprachen', 'Ungarisch', 'Isländisch',
       'Übrige westasiatische Sprachen', 'Tschechoslowakische Sprachen',
       'Bulgarisch', 'Somali', 'Koreanisch', 'Übrige westeurop. Sprach

In [18]:
#jahre?
df_all["Jahr"].unique()

array([2002, 2016, 2003, 2011, 2004, 2010, 2005, 2009, 2013, 2006, 2012,
       2007, 2008, 2015, 2000, 2017, 2014, 2001])

In [19]:
# liste mit den gemeinden machen, die mich interessieren (zürichsee)
gemeinden = ["Adliswil (Einheitsgemeinde)",
             "Erlenbach (Einheitsgemeinde)",
             "Herrliberg (Einheitsgemeinde)",
             "Hirzel (Einheitsgemeinde)",
             "Hombrechtikon (Einheitsgemeinde)",
             "Horgen (Einheitsgemeinde)",
             "Hütten (Primar)",
             "Kilchberg (Einheitsgemeinde)",
             "Küsnacht (Primar + Sekundar)",
             "Langnau a.A. (Einheitsgemeinde)",
             "Männedorf (Einheitsgemeinde)",
             "Meilen (Einheitsgemeinde)",
             "Oberrieden (Einheitsgemeinde)",
             "Oetwil a.S. (Einheitsgemeinde)",
             "Richterswil (Einheitsgemeinde)",
             "Rüschlikon (Einheitsgemeinde)",
             "Schönenberg (Primar + Pol. Gem.)",
             "Stäfa (Primar + Sekundar)",
             "Thalwil (Einheitsgemeinde)",
             "Uetikon a.S. (Einheitsgemeinde)",
             "Wädenswil (Primar + Pol. Gem.)",
             "Wädenswil (Sekundar)",
             "Zollikon (Einheitsgemeinde)",
             "Zumikon (Primar + Sekundar)",
             "Erlenbach-Herrliberg (Sek.schul Verband)",
             "Hütten (Primar + Pol. Gem.)",
             "Kilchberg-Rüschlikon (Sek.schul Verband)",
             "Stäfa (Einheitsgemeinde)",
             "Zumikon (Einheitsgemeinde)"
            ]

In [20]:
# neues df schaffen nur mit "meinen" gemeinden
df_all[df_all["Schulgemeinde"].isin(gemeinden)]

,Jahr,Schulgemeinde,Erstsprache,KG,PS,PS Sonder,Sek A,Sek B,Sek C,Sek Sonder,Total
1,2002,Adliswil (Einheitsgemeinde),Afrikanische Sprachen,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3
2,2002,Adliswil (Einheitsgemeinde),Albanisch,12.0,63.0,10.0,4.0,8.0,2.0,4.0,103
3,2002,Adliswil (Einheitsgemeinde),Andere asiatische Sprachen,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2
4,2002,Adliswil (Einheitsgemeinde),Arabisch,1.0,3.0,2.0,1.0,0.0,0.0,1.0,8
5,2002,Adliswil (Einheitsgemeinde),Bosnisch,3.0,3.0,1.0,0.0,0.0,0.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...
74435,2001,Zumikon (Primar + Sekundar),Türkisch,0.0,4.0,0.0,0.0,1.0,0.0,0.0,5
74436,2001,Zumikon (Primar + Sekundar),Übrige afrikanische Sprachen,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
74437,2001,Zumikon (Primar + Sekundar),Übrige europäische Sprachen,2.0,1.0,0.0,0.0,0.0,0.0,0.0,3
74438,2001,Zumikon (Primar + Sekundar),Übrige Sprachen,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1


In [21]:
df_zuerichsee = df_all[df_all["Schulgemeinde"].isin(gemeinden)]

In [22]:
#test
df_zuerichsee.head(10)

,Jahr,Schulgemeinde,Erstsprache,KG,PS,PS Sonder,Sek A,Sek B,Sek C,Sek Sonder,Total
1,2002,Adliswil (Einheitsgemeinde),Afrikanische Sprachen,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3
2,2002,Adliswil (Einheitsgemeinde),Albanisch,12.0,63.0,10.0,4.0,8.0,2.0,4.0,103
3,2002,Adliswil (Einheitsgemeinde),Andere asiatische Sprachen,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2
4,2002,Adliswil (Einheitsgemeinde),Arabisch,1.0,3.0,2.0,1.0,0.0,0.0,1.0,8
5,2002,Adliswil (Einheitsgemeinde),Bosnisch,3.0,3.0,1.0,0.0,0.0,0.0,0.0,7
6,2002,Adliswil (Einheitsgemeinde),Chinesisch,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2
7,2002,Adliswil (Einheitsgemeinde),Deutsch,180.0,512.0,33.0,126.0,109.0,6.0,8.0,974
8,2002,Adliswil (Einheitsgemeinde),Englisch,5.0,16.0,1.0,3.0,0.0,0.0,0.0,25
9,2002,Adliswil (Einheitsgemeinde),Finnisch,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
10,2002,Adliswil (Einheitsgemeinde),Französisch,3.0,9.0,1.0,3.0,1.0,0.0,1.0,18


In [23]:
df_zuerichsee.dtypes

Jahr               int64
Schulgemeinde     object
Erstsprache       object
KG               float64
PS               float64
PS Sonder        float64
Sek A            float64
Sek B            float64
Sek C            float64
Sek Sonder       float64
Total              int64
dtype: object

In [24]:
df_zuerichsee.info

<bound method DataFrame.info of        Jahr                Schulgemeinde                   Erstsprache    KG  \
1      2002  Adliswil (Einheitsgemeinde)         Afrikanische Sprachen   1.0   
2      2002  Adliswil (Einheitsgemeinde)                     Albanisch  12.0   
3      2002  Adliswil (Einheitsgemeinde)    Andere asiatische Sprachen   0.0   
4      2002  Adliswil (Einheitsgemeinde)                      Arabisch   1.0   
5      2002  Adliswil (Einheitsgemeinde)                      Bosnisch   3.0   
...     ...                          ...                           ...   ...   
74435  2001  Zumikon (Primar + Sekundar)                      Türkisch   0.0   
74436  2001  Zumikon (Primar + Sekundar)  Übrige afrikanische Sprachen   1.0   
74437  2001  Zumikon (Primar + Sekundar)   Übrige europäische Sprachen   2.0   
74438  2001  Zumikon (Primar + Sekundar)               Übrige Sprachen   0.0   
74439  2001  Zumikon (Primar + Sekundar)                     Ungarisch   0.0   

       

In [25]:
#sortierung testen
df_zuerichsee.sort_values('Total', ascending=False).head(10)

,Jahr,Schulgemeinde,Erstsprache,KG,PS,PS Sonder,Sek A,Sek B,Sek C,Sek Sonder,Total
73827,2001,Wädenswil (Primar + Pol. Gem.),Deutsch,304.0,849.0,48.0,NaN,NaN,NaN,NaN,1201
2752,2002,Wädenswil (Primar + Pol. Gem.),Deutsch,294.0,866.0,40.0,NaN,NaN,NaN,NaN,1200
61370,2000,Wädenswil (Primar + Pol. Gem.),Deutsch,276.0,846.0,44.0,NaN,NaN,NaN,NaN,1166
11137,2003,Wädenswil (Primar + Pol. Gem.),Deutsch,265.0,847.0,52.0,NaN,NaN,NaN,NaN,1164
10418,2003,Richterswil (Einheitsgemeinde),Deutsch,196.0,657.0,12.0,155.0,114.0,12.0,0.0,1146
2078,2002,Richterswil (Einheitsgemeinde),Deutsch,196.0,672.0,9.0,147.0,107.0,11.0,0.0,1142
18689,2004,Richterswil (Einheitsgemeinde),Deutsch,196.0,645.0,18.0,152.0,114.0,12.0,0.0,1137
73179,2001,Richterswil (Einheitsgemeinde),Deutsch,203.0,656.0,7.0,144.0,115.0,10.0,0.0,1135
60782,2000,Richterswil (Einheitsgemeinde),Deutsch,207.0,672.0,3.0,150.0,91.0,6.0,0.0,1129
19428,2004,Wädenswil (Primar + Pol. Gem.),Deutsch,260.0,820.0,48.0,NaN,NaN,NaN,NaN,1128


In [26]:
#index auf jahr setzen
#df_zuerichsee.set_index('Jahr', inplace=True)

In [62]:
#df_zuerichsee

In [63]:
#Erstsprache als index definieren
df_zuerichsee.set_index("Erstsprache", inplace=True)

In [64]:
#test
df_zuerichsee

,Jahr,Schulgemeinde,KG,PS,PS Sonder,Sek A,Sek B,Sek C,Sek Sonder,Total
Erstsprache,,,,,,,,,,
Afrikanische Sprachen,2002,Adliswil (Einheitsgemeinde),1.0,2.0,0.0,0.0,0.0,0.0,0.0,3
Albanisch,2002,Adliswil (Einheitsgemeinde),12.0,63.0,10.0,4.0,8.0,2.0,4.0,103
Andere asiatische Sprachen,2002,Adliswil (Einheitsgemeinde),0.0,1.0,0.0,1.0,0.0,0.0,0.0,2
Arabisch,2002,Adliswil (Einheitsgemeinde),1.0,3.0,2.0,1.0,0.0,0.0,1.0,8
Bosnisch,2002,Adliswil (Einheitsgemeinde),3.0,3.0,1.0,0.0,0.0,0.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...
Türkisch,2001,Zumikon (Primar + Sekundar),0.0,4.0,0.0,0.0,1.0,0.0,0.0,5
Übrige afrikanische Sprachen,2001,Zumikon (Primar + Sekundar),1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
Übrige europäische Sprachen,2001,Zumikon (Primar + Sekundar),2.0,1.0,0.0,0.0,0.0,0.0,0.0,3


In [29]:
df_pivoted = df_zuerichsee.pivot(index='Erstsprache', columns='Jahr', values='Total')

ValueError: Index contains duplicate entries, cannot reshape

In [28]:
df_pivoted

Jahr,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Erstsprache,,,,,,,,,,,,,,,,,,
Afrikanische Sprachen,2.090909,1.545455,1.833333,1.727273,1.800000,1.500000,1.300000,1.333333,1.625000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN
Albanisch,45.347826,45.000000,47.130435,48.304348,50.260870,52.000000,52.826087,48.375000,48.958333,48.666667,49.208333,48.416667,48.583333,49.458333,53.652174,48.920000,46.884615,46.653846
Andere asiatische Sprachen,3.250000,3.117647,3.263158,4.055556,5.200000,5.714286,4.235294,4.533333,4.583333,3.545455,2.750000,2.000000,1.400000,1.250000,1.000000,1.000000,NaN,NaN
Arabisch,2.736842,3.647059,3.937500,4.055556,4.277778,4.687500,4.500000,4.111111,4.235294,4.375000,3.944444,5.052632,5.157895,5.764706,6.941176,7.578947,9.363636,10.863636
Bosnisch,2.166667,3.083333,3.071429,3.142857,3.846154,3.866667,4.538462,5.250000,5.666667,5.076923,6.000000,6.400000,7.500000,7.090909,6.230769,6.916667,6.833333,5.857143
Bulgarisch,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.200000,1.333333,1.285714,1.777778,1.875000,2.444444,2.666667,2.600000,2.166667,2.307692
Chinesisch,1.250000,1.125000,1.833333,1.416667,1.333333,1.800000,1.933333,2.000000,2.352941,2.687500,2.588235,2.500000,2.882353,2.789474,3.111111,3.777778,3.736842,3.857143
Deutsch,610.625000,610.583333,604.666667,603.416667,597.791667,591.291667,580.750000,548.640000,541.600000,536.680000,528.160000,520.640000,516.520000,511.600000,511.080000,497.615385,509.076923,513.576923
Dänisch,1.142857,1.454545,1.777778,1.700000,2.000000,2.000000,2.083333,2.538462,2.583333,3.000000,2.705882,3.571429,3.846154,3.437500,3.812500,3.705882,3.368421,3.166667


In [65]:
#ziele: ich möchte die entwicklung der einzelnen sprachgruppen ab 2000 für jede einzelne gemeinde
#darstellen und basierend auf den resultaten weitere schlüsse ziehen, resp. meine these beweisen (abnahme)
#der alten migrationsgruppen (balkan, italien, spanien), zunahme der neuen (englisch, französisch,)

Problem: Ich habe 18 Files und pro Sprache und Gemeinde jeweils eine separate jahresangabe? 